## Loading the data
## Clean the data
## Feature Enginnering
## Modelling
## Save model

<hr>

In [1]:
## Load the data

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('./data.csv')

In [4]:
## Clean the data

In [5]:
data.columns

Index(['date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'street', 'city',
       'statezip', 'country'],
      dtype='object')

In [6]:
data.drop(['sqft_living','sqft_lot','waterfront','view','condition','sqft_above','sqft_basement','street','city','statezip','country'],axis=1,inplace=True)

In [7]:
data.drop('date',axis=1,inplace=True)

In [8]:
data.head()

,price,bedrooms,bathrooms,floors,yr_built,yr_renovated
0,313000.0,3.0,1.50,1.5,1955,2005
1,2384000.0,5.0,2.50,2.0,1921,0
2,342000.0,3.0,2.00,1.0,1966,0
3,420000.0,3.0,2.25,1.0,1963,0
4,550000.0,4.0,2.50,1.0,1976,1992


In [9]:
## Feature Enginnering

In [10]:
def fe(data,col):
    print(len(data))
    max_no = data[col].quantile(0.99)
    min_no = data[col].quantile(0.05)
    data = data[data[col] > min_no]
    data = data[data[col] < max_no]
    print(len(data))
    return data

In [11]:
for col in list(data.columns):
    print(col)
    data = fe(data,'price')

price
4600
4316
bedrooms
4316
4056
bathrooms
4056
3808
floors
3808
3578
yr_built
3578
3362
yr_renovated
3362
3156


In [12]:
data.head()

,price,bedrooms,bathrooms,floors,yr_built,yr_renovated
2,342000.0,3.0,2.00,1.0,1966,0
3,420000.0,3.0,2.25,1.0,1963,0
4,550000.0,4.0,2.50,1.0,1976,1992
5,490000.0,2.0,1.00,1.0,1938,1994
6,335000.0,2.0,2.00,1.0,1976,0


In [13]:
X = data.drop('price',axis=1)
y = data['price']

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [16]:
len(X_train),len(X_test)

(2367, 789)

In [17]:
## Modelling

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim

In [19]:
import torch.nn.functional as F

In [20]:
class BaseLine_Model(nn.Module):
    def __init__(self,input_shape,output_shape):
        super().__init__()
        self.fc1 = nn.Linear(input_shape,64)
        self.fc2 = nn.Linear(64,128)
        self.fc3 = nn.Linear(128,256)
        self.fc4 = nn.Linear(256,128)
        self.fc5 = nn.Linear(128,output_shape)
    
    def forward(self,X):
        preds = self.fc1(X)
        preds = F.relu(preds)
        preds = self.fc2(preds)
        preds = F.relu(preds)
        preds = self.fc3(preds)
        preds = F.relu(preds)
        preds = self.fc4(preds)
        preds = F.relu(preds)
        preds = self.fc5(preds)
        return preds

In [21]:
import wandb

In [22]:
BATCH_SIZE = 32

In [23]:
PROJECT_NAME = 'House-Price-Pred'

In [24]:
from tqdm import tqdm

In [25]:
device = torch.device('cuda')

In [26]:
model = BaseLine_Model(5,1).to(device)

In [27]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=0.1)

In [28]:
def get_loss(criterion,X,y,model):
    preds = model(X.float().to(device))
    preds = preds.to(device)
    y = y.to(device)
#     criterion.to(device)
    loss = criterion(preds,y)
    return loss.item()
def get_accuracy(X,y,model):
    correct = 0
    total = 0
    for i in range(len(X)):
        pred = model(X[i].float().to(device))
        pred.to(device)
        if round(int(pred[0])) == round(int(y[i])):
            correct += 1
        total += 1
    if correct == 0:
        correct += 1
    return round(correct/total,3)

In [29]:
import numpy as np

In [30]:
X_train = torch.from_numpy(np.array(X_train))
y_train = torch.from_numpy(np.array(y_train))
X_test = torch.from_numpy(np.array(X_test))
y_test = torch.from_numpy(np.array(y_test))

In [31]:
get_accuracy(X_test,y_test,model)

0.001

In [32]:
EPOCHS = 112

In [33]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    model.to(device)
    preds = model(X_train.float())
    preds = preds.view(len(preds))
    preds.to(device)
    loss = criterion(preds.float(),y_train.float())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)


  0%|          | 0/112 [00:00<?, ?it/s]


NameError: name 'i' is not defined

In [ ]:
preds

In [ ]:
y_train

In [ ]:
torch.round(preds)

In [ ]:
torch.round(y_train)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
preds[0]

In [ ]:
y_train[0]